In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt


In [ ]:

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:

Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
Base.classes.keys()

In [ ]:
Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:

columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])


In [ ]:

engine.execute("select * from Measurement limit 10").fetchall()
session.query(func.count(Measurement.date)).all()



current_year=session.query(func.max(Measurement.date)).all()
current_year

val_cal = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= "2016-08-23").\
        filter(Measurement.date <= "2017-08-23").\
        group_by(Measurement.date).all()
val_cal


df = pd.DataFrame(val_cal, columns=['date','prcp'])

df.sort_values(by='date',ascending= True) 
df.rename(columns={0:"Date",1:"Precipitation"},inplace =True)
df.set_index('date')
df.head

df.plot()
plt.figure(1)
df.describe()

In [ ]:


total_stations = session.query(func.count(Station.station)).all()
total_stations

In [ ]:


active_stations = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
active_stations


active_stations_val =active_stations[0] 
active_stations_val

In [ ]:



temp_observation = session.query(Measurement.station, func.count(Measurement.station)).\
                group_by(Measurement.station).\
                order_by(func.count(Measurement.station).desc()).all()
temp_observation

In [ ]:

all_observations = session.query(Measurement.station,func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).filter(Measurement.station == active_stations_val[0]).all()
all_observations

In [ ]:


active_stations_val =active_stations[0] 
active_stations_val

active_stations_temps= session.query(Measurement.tobs).\
                        filter(Measurement.date >= "2016-08-23").\
                       filter(Measurement.station == "USC00519281").all()
active_stations_temps
len(active_stations_temps)

all_temps= [z[0] for z in active_stations_temps]
all_temps

In [ ]:

plt.hist(all_temps, bins=12, color="blue")
plt.title("Displaying 12 months temp of highest station")
labels = ['tobs']
plt.ylabel('Frequency', weight='bold')
plt.xlabel('Temperature', weight='bold')
plt.figure(2)

In [ ]:

start_date ="2017-01-01" 
end_date= "2018-01-01"
def calc_temps(start_date, end_date):
    cal_btwn_dates= session.query(Measurement.tobs).\
                     filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
    values_dates= [k[0] for k in cal_btwn_dates]
    avg_temp=np.mean(values_dates)
    minimum_temp=min(values_dates)
    maximum_temp=max(values_dates)
    print(avg_temp)
    print(minimum_temp)
    print(maximum_temp)
print(calc_temps('2012-02-28', '2012-03-05'))

plt.bar(1,height = 69.57, yerr = 8, align='center', color="coral")


plt.title("Trip Avg Temp")
plt.ylabel("Temp(F)")
plt.figure(3)
plt.show()

In [ ]:
active_stations_val =active_stations 
active_stations_val

station_list = []
for g in active_stations_val:
    station_list.append(g)

all_stations = session.query(Measurement.station, func.sum(Measurement.prcp), func.sum(Measurement.tobs), Station.name, Station.latitude, Station.longitude, Station.elevation).\
                group_by(Measurement.station).\
                order_by(func.sum(Measurement.prcp).desc()).all()
all_stations

In [ ]:


date="01-01"
def daily_normals(date):
    daily_normals("01-01")
    
sell = [func.min(Measurement.tobs), 
           func.avg(Measurement.tobs), 
           func.max(Measurement.tobs)]

daily_nor = session.query(* sell).\
            filter(func.strftime("%m-%d", Measurement.date) == date).all()
daily_nor

In [ ]:



start = "2017-01-01"
end = "2017-01-07"

def daily_normals(start,end):
    daily_normals_date = session.query(Measurement.tobs).filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    daily_dates = [l[0] for l in daily_normals_date]
    avgg_temps = np.mean(daily_dates)
    min_temps = min(daily_dates)
    max_temps = max(daily_dates)   
    print(avgg_temps)
    print(min_temps)
    print(max_temps)
print(daily_normals("2017-01-01","2017-01-07"))

aaa= pd.DataFrame(daily_normals_date)
aaa.head()

